In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/semajyllek/ctmatch.git

Cloning into 'ctmatch'...
remote: Enumerating objects: 791, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 791 (delta 147), reused 162 (delta 67), pack-reused 545
Receiving objects: 100% (791/791), 318.36 KiB | 10.61 MiB/s, done.
Resolving deltas: 100% (492/492), done.


In [4]:
!pip install -r /content/ctmatch/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/semajyllek/transformers.git (to revision add-biogpt-sequenceclassifier) to /tmp/pip-req-build-d4kez2u8
  Running command git clone --filter=blob:none --quiet https://github.com/semajyllek/transformers.git /tmp/pip-req-build-d4kez2u8
  Running command git checkout -b add-biogpt-sequenceclassifier --track origin/add-biogpt-sequenceclassifier
  Switched to a new branch 'add-biogpt-sequenceclassifier'
  Branch 'add-biogpt-sequenceclassifier' set up to track remote branch 'add-biogpt-sequenceclassifier' from 'origin'.
  Resolved https://github.com/semajyllek/transformers.git to commit e9b0da2f391f4a140d5cb794c5a3b7ea4c5c2fad
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import sys
sys.path.insert(1, '/content/ctmatch');sys.path

['/content',
 '/content/ctmatch',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [14]:
from pathlib import Path
TREC_DATA_PATH = Path('/content/drive/MyDrive/ct_data23/trec_data.jsonl')
KZ_DATA_PATH = Path('/content/drive/MyDrive/ct_data23/kz_data.jsonl')
COMBINED_DATA_PATH = Path('/content/drive/MyDrive/ct_data23/combined_classifier_data.jsonl')

In [8]:
from ctmatch.match import ModelConfig, CTMatch

In [9]:
from transformers import logging
logging.set_verbosity_error()

In [15]:
nli_model = 'cross-encoder/nli-roberta-base'
scibert_model = 'allenai/scibert_scivocab_uncased'
biogpt_model = 'microsoft/biogpt'
bart_model = 'facebook/bart-large'
gpt2_model = 'gpt2'
model_name = scibert_model


model_config = ModelConfig(
    name=f'ctmatch_finetuned_{model_name}-classifier',
    data_path=COMBINED_DATA_PATH,
    model_checkpoint=model_name,
    max_length=512,
    batch_size=8,
    padding='max_length',
    truncation=True,
    learning_rate=2e-5,
    train_epochs=6,
    weight_decay=0.01,
    warmup_steps=500,
    seed=42,
    splits={"train":0.8, "val":0.1},
    output_dir = f"/content/drive/MyDrive/finetuned_{model_name.split('/')[-1]}model",
    use_trainer=True,
    convert_snli=False,
    fp16=True,
    early_stopping=True,
    push_to_hub=True
)

ctm = CTMatch(model_config)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ef0715b88f8e3848/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/28576 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/28576 [00:00<?, ? examples/s]

Map:   0%|          | 0/7939 [00:00<?, ? examples/s]

Map:   0%|          | 0/3176 [00:00<?, ? examples/s]

Using device: cuda


Cloning https://huggingface.co/semaj83/finetuned_scibert_scivocab_uncasedmodel into local empty directory.


In [16]:
ctm.classifier_model.dataset

DatasetDict({
    train: Dataset({
        features: ['doc', 'labels', 'topic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 28576
    })
    test: Dataset({
        features: ['doc', 'labels', 'topic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7939
    })
    validation: Dataset({
        features: ['doc', 'labels', 'topic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3176
    })
})

In [17]:
ctm.classifier_model.dataset['train'].features

{'doc': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['not_relevant', 'partially_relevant', 'relevant'], id=None),
 'topic': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [18]:
ctm.classifier_model.tokenizer.decode(ctm.classifier_model.dataset['train'][0]['input_ids'])

"[CLS] a 22 - year - old caucasian man came to the clinic with a history of tremors since a year ago. the tremor was first in his right hand while holding something. later the tremor became continuous and extended to both hands and legs and even at rest. the kayser - fleischer'ring was detected in the ophthalmologic exam. the physical exam revealed jaundice, hepatosplenomegaly and hypotonia of the upper limbs. he had a constant smile on his face, however, he has aggressive behavior according to his parents'explanation. his laboratory study was significant for a low serum caeruloplasmin ( 0. 05 g / l ), and a raised 24 hour urine copper excretion ( 120 μg / 24 h ). wilson disease was confirmed by high liver copper concentration ( 305 μg / g dry weight of liver ). [SEP] eligible ages ( years ) : 18. 0 - 70. 0, efficacy and safety ( 1 ) 18 years < the age of patients > 70 years. ( 2 ) karnofsky performance scale ( kps ) ≥ 60. ( 2 ) histologically confirmed diagnosis of gbm world health or

In [19]:
import torch
torch.cuda.empty_cache()

| model | f1 | epochs | max input length | batch size |
|-------|--------|--------|------------------|------------|
| bart | 79.5 | 7 | 512 | 8 |
| scibert |81.1 |4 |512 | 16 |
| nli-roberta | 79.8 | 4 | 512 | 8 |
| biogpt | 78 | 5 | 512 | 8 |

In [14]:
#manual_train(ctm.classifier_model)

In [20]:
ctm.classifier_model.train_and_predict()

{'loss': 0.6188, 'learning_rate': 1.9640609279580715e-05, 'epoch': 1.0}
{'eval_loss': 0.5282624363899231, 'eval_f1': 0.7567711957161386, 'eval_runtime': 24.5619, 'eval_samples_per_second': 129.306, 'eval_steps_per_second': 16.163, 'epoch': 1.0}
{'loss': 0.5036, 'learning_rate': 1.922272397575985e-05, 'epoch': 2.0}
{'eval_loss': 0.5223625302314758, 'eval_f1': 0.778970313266787, 'eval_runtime': 24.5328, 'eval_samples_per_second': 129.459, 'eval_steps_per_second': 16.182, 'epoch': 2.0}
{'loss': 0.448, 'learning_rate': 1.880483867193898e-05, 'epoch': 3.0}
{'eval_loss': 0.5717465281486511, 'eval_f1': 0.7859790826452602, 'eval_runtime': 24.5354, 'eval_samples_per_second': 129.446, 'eval_steps_per_second': 16.181, 'epoch': 3.0}
{'loss': 0.391, 'learning_rate': 1.8386953368118113e-05, 'epoch': 4.0}
{'eval_loss': 0.5893653631210327, 'eval_f1': 0.7837758533086763, 'eval_runtime': 24.5213, 'eval_samples_per_second': 129.52, 'eval_steps_per_second': 16.19, 'epoch': 4.0}
{'loss': 0.3437, 'learning_

In [16]:
cm, cr = ctm.classifier_model.get_sklearn_metrics()

In [17]:
print(cm)

[[1738   96  120]
 [  93  229  135]
 [  79   92  284]]


In [18]:
print(cr)

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      1954
           1       0.55      0.50      0.52       457
           2       0.53      0.62      0.57       455

    accuracy                           0.79      2866
   macro avg       0.66      0.67      0.67      2866
weighted avg       0.79      0.79      0.79      2866

